In [ ]:
t = [[0, 1, 2, 3, 4], [6, 7, 8, 9]]
[_ for _ in test(t)]

In [1]:
import json

In [3]:
def test(text):
    for JSONobject in text["cdl"]:
        if "cdl" in JSONobject:
            for x in test(JSONobject):
                continue
        elif "f" in JSONobject:
            word = JSONobject["f"]
            yield word.get("cf", "")

In [3]:
import zipfile
import json
file = "jsonzip/epsd2-admin-ur3.zip"
z = zipfile.ZipFile(file)
filename = "epsd2/admin/ur3/gloss-sux.json"
with open(file, "r", encoding="utf8") as r:
    data = z.read(filename).decode('utf-8')         #read and decode the json file of one particular text
data_json = json.loads(data)                # make it into a json object (essentially a dictionary)


In [22]:
entries = data_json["entries"]
x2oid = {entry.get("headword", "") : entry.get("oid", "") for entry in entries}

In [24]:
for key in x2oid: 
    if x2oid[key] == "": 
        print(key)

gugumuʾe[rite]N
n[n]n


In [ ]:
type(p)

In [ ]:
p1 = list(p[1])

In [ ]:
p2 = list(p[2])

In [ ]:
p3 = list(p[3])

In [16]:
from scipy import spatial

a = [3, 45, 7, 2]
b = [2, 54, 13, 15]
result = 1 - spatial.distance.cosine(a, b)

In [11]:
result

0.97228425171235

In [17]:
from numpy.linalg import norm
import numpy as np

In [19]:
a = np.array(a)
b = np.array(b)
cos_sim = np.dot(a, b)/(norm(a)*norm(b))

In [20]:
cos_sim

0.9722842517123499